In [1]:
import pandas as pd
import sqlite3

In [2]:
#Leitura do arquivo inicial
dados = pd.read_excel('META.xlsx',header=None)

In [3]:
dados.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Código,Vendedor/Ano,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro,NaN,Erro?,0
1,METAS,2017,4309000,4274000,4208000,4690000,4608000,4403000,4306000,4498000,4478000,4748000,4556000,4603000,NaN,Brancos,Códigos
2,44,Vendedor 44,320000,0,320000,320000,320000,320000,320000,320000,320000,320000,320000,320000,NaN,0,0
3,92,Vendedor 92,0,263000,263000,263000,263000,263000,263000,263000,263000,263000,263000,263000,NaN,0,0
4,33,Vendedor 33,170000,170000,170000,170000,170000,170000,170000,170000,170000,170000,170000,170000,NaN,0,0


In [4]:
#deletando linhas e colunas desnecessárias 
dados.drop([14,15,16],axis=1,inplace=True)
dados.dropna(inplace=True)

In [5]:
meta = pd.DataFrame()
size = len(dados)

In [6]:
#Criação da nova tabela
for i in range(2,size):
    for j in range(2,14):
        anometa = dados[1][1]
        mesmeta = dados[j][0]
        vendedorcodigo = dados[0][i]
        vendedornome = dados[1][i]
        valormeta = dados[j][i]
        meta = meta.append([[anometa, mesmeta, vendedorcodigo, vendedornome, valormeta]])
        
meta.columns=['anometa','mesmeta','vendedorcodigo','vendedornome','valormeta']

In [7]:
meta.head()

,anometa,mesmeta,vendedorcodigo,vendedornome,valormeta
0,2017,Janeiro,44,Vendedor 44,320000
0,2017,Fevereiro,44,Vendedor 44,0
0,2017,Março,44,Vendedor 44,320000
0,2017,Abril,44,Vendedor 44,320000
0,2017,Maio,44,Vendedor 44,320000


In [8]:
# Conexão com o SQLite
DB = sqlite3.connect('bancolocal.db')

In [9]:
#Criação da tabela no SQLite

c = DB.cursor()

c.execute(''' CREATE TABLE meta (anometa smallint, 
                                mesmeta smallint, 
                                vendedorcodigo varchar(255), 
                                vendedornome varchar(255), 
                                valormeta double precision); ''')


In [10]:
#Migração do pandas para o SQLite
meta.to_sql('meta', con=DB, if_exists='append', index=False)

In [11]:
c.execute(''' SELECT * from meta ''')
c.fetchone()

(2017, 'Janeiro', '44', 'Vendedor 44', 320000.0)

In [12]:
#sql query
c.execute(''' SELECT vendedorcodigo, vendedornome,
                SUM(CASE WHEN mesmeta = 'Janeiro' THEN "valormeta" END) AS Janeiro,
                SUM(CASE WHEN mesmeta = 'Fevereiro' THEN "valormeta" END) AS Fevereiro,
                SUM(CASE WHEN mesmeta = 'Março' THEN "valormeta" END) AS Março,
                SUM(CASE WHEN mesmeta = 'Abril' THEN "valormeta" END) AS Abril,
                SUM(CASE WHEN mesmeta = 'Maio' THEN "valormeta" END) AS Maio,
                SUM(CASE WHEN mesmeta = 'Junho' THEN "valormeta" END) AS Junho,
                SUM(CASE WHEN mesmeta = 'Julho' THEN "valormeta" END) AS Julho,
                SUM(CASE WHEN mesmeta = 'Agosto' THEN "valormeta" END) AS Agosto,
                SUM(CASE WHEN mesmeta = 'Setembro' THEN "valormeta" END) AS Setembro,
                SUM(CASE WHEN mesmeta = 'Outubro' THEN "valormeta" END) AS Outubro,
                SUM(CASE WHEN mesmeta = 'Novembro' THEN "valormeta" END) AS Novembro,
                SUM(CASE WHEN mesmeta = 'Dezembro' THEN "valormeta" END) AS Dezembro
                from meta
                GROUP BY vendedorcodigo''')
header = [d[0] for d in c.description]
table = c.fetchall()

In [13]:
DB.commit()
DB.close()

In [14]:
# Migração da query do SQLite para o Pandas
resultado = pd.DataFrame(table,columns=header)

In [16]:
resultado.head()

,vendedorcodigo,vendedornome,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,114,Vendedor 114,0.0,356000.0,356000.0,356000.0,356000.0,356000.0,356000.0,356000.0,356000.0,356000.0,356000.0,356000.0
1,140,Vendedor 140,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,180000.0,0.0,180000.0,180000.0,180000.0,180000.0
2,146,Vendedor 146,315000.0,315000.0,0.0,315000.0,315000.0,315000.0,315000.0,315000.0,315000.0,315000.0,315000.0,315000.0
3,153,Vendedor 153,250000.0,250000.0,250000.0,250000.0,250000.0,250000.0,0.0,250000.0,250000.0,250000.0,250000.0,250000.0
4,155,Vendedor 155,238000.0,238000.0,238000.0,0.0,238000.0,238000.0,238000.0,238000.0,238000.0,238000.0,238000.0,238000.0


In [15]:
#Escrita do csv final
resultado.to_csv('meta.csv',index=False)